In [9]:
# Generated synthetic univariate or multivariate time series data
import pandas as pd
import numpy as np

def generate_time_series_data(start_time: str = "2021-01-01 00:00:00", 
                              freq: str = "H", 
                              periods: int = 40, 
                              n_variables: int = 10, 
                              univariate: bool = False) -> pd.DataFrame:
    """
    Generate a univariate or multivariate time series DataFrame.
    
    Args:
        start_time (str): Start datetime.
        freq (str): Pandas frequency string (e.g., 'H' for hourly).
        periods (int): Number of time steps.
        n_variables (int): Number of variables (ignored if univariate=True).
        univariate (bool): Whether to generate univariate (single-column) data.

    Returns:
        pd.DataFrame: Time series data with datetime index.
    """
    index = pd.date_range(start=start_time, periods=periods, freq=freq)

    if univariate:
        data = np.random.randn(periods)
        df = pd.DataFrame(data, index=index, columns=["A"])
    else:
        columns = list("ABCDEFGHIJ")[:n_variables]
        data = np.random.randn(periods, n_variables)
        df = pd.DataFrame(data, index=index, columns=columns)
    
    return df

df_multivariate = generate_time_series_data()
print(df_multivariate.columns.values)
df_univariate = generate_time_series_data(univariate=True)
print(df_univariate.head())

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J']
                            A
2021-01-01 00:00:00  0.407823
2021-01-01 01:00:00  0.042234
2021-01-01 02:00:00  1.757940
2021-01-01 03:00:00 -0.227974
2021-01-01 04:00:00 -1.761402


In [10]:
import torch
import matplotlib.pyplot as plt
import pandas as pd
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from huggingface_hub import hf_hub_download

from uni2ts.eval_util.plot import plot_single
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.model.moirai_moe import MoiraiMoEForecast, MoiraiMoEModule

# Model configuration
MODEL = "moirai"  # model name: choose from {'moirai', 'moirai_moe'}
SIZE = "small"  # model size: choose from {'small', 'base', 'large'}
PDT = 4  # prediction length: any positive integer
CTX = 10  # context length: any positive integer
PSZ = 8  # patch size: choose from {"auto", 8, 16, 32, 64, 128}
BSZ = 8  # batch size: any positive integer
TEST = 8  # test set length: any positive integer

gluon_multi = PandasDataset(dict(df_multivariate))
gluon_uni = PandasDataset(dict(df_univariate))
print(gluon_multi)
print(gluon_uni)

PandasDataset<size=10, freq=H, num_feat_dynamic_real=0, num_past_feat_dynamic_real=0, num_feat_static_real=0, num_feat_static_cat=0, static_cardinalities=[]>
PandasDataset<size=1, freq=H, num_feat_dynamic_real=0, num_past_feat_dynamic_real=0, num_feat_static_real=0, num_feat_static_cat=0, static_cardinalities=[]>


In [11]:
# Obtain train and test data
train_multi, test_template_multi = split(
    gluon_multi, offset=-TEST
)  # assign last TEST time steps as test set


train_uni, test_template_uni = split(
    gluon_uni, offset=-TEST
)  # assign last TEST time steps as test set


# Construct rolling window evaluation
test_data_multi = test_template_multi.generate_instances(
    prediction_length=PDT,  # number of time steps for each prediction
    windows=TEST // PDT,  # number of windows in rolling window evaluation
    distance=PDT,  # number of time steps between each window - distance=PDT for non-overlapping windows
)

# Construct rolling window evaluation
test_data_uni = test_template_uni.generate_instances(
    prediction_length=PDT,  # number of time steps for each prediction
    windows=TEST // PDT,  # number of windows in rolling window evaluation
    distance=PDT,  # number of time steps between each window - distance=PDT for non-overlapping windows
)



In [12]:
def moirai_maker(MODEL, gluon_data):
    if MODEL == "moirai":
        model = MoiraiForecast(
            module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.1-R-{SIZE}"),
            prediction_length=PDT,
            context_length=CTX,
            patch_size=PSZ,
            num_samples=5,
            target_dim=1,
            feat_dynamic_real_dim=gluon_data.num_feat_dynamic_real,
            past_feat_dynamic_real_dim=gluon_data.num_past_feat_dynamic_real,
        )
    elif MODEL == "moirai_moe":
        model = MoiraiMoEForecast(
            module=MoiraiMoEModule.from_pretrained(f"Salesforce/moirai-moe-1.0-R-{SIZE}"),
            prediction_length=PDT,
            context_length=CTX,
            patch_size=PSZ,
            num_samples=5,
            target_dim=1,
            feat_dynamic_real_dim=gluon_data.num_feat_dynamic_real,
            past_feat_dynamic_real_dim=gluon_data.num_past_feat_dynamic_real,
        )
    return model

In [13]:
# Moirai_MoE multivariate
model = moirai_maker("moirai_moe", gluon_multi)
predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data_multi.input)

input_it = iter(test_data_multi.input)
label_it = iter(test_data_multi.label)
forecast_it = iter(forecasts)

#inp = next(input_it)
#label = next(label_it)
#forecast = next(forecast_it)
results_dict = dict()
letter_sequence = "ABCDEFGHIJ"
for letter in letter_sequence:
    results_dict[letter] = []
    
for fore in forecast_it:
    #print(fore)
    for letter in letter_sequence:
        if fore.item_id == letter:
            #print(fore.samples)
            results_dict[letter].extend(np.median(fore.samples,axis=0))


print(results_dict)

{'A': [0.22621918, 0.22547217, 0.22547217, 1.2673733, 0.94532037, 0.0027131368, 0.5578801, 0.77568424], 'B': [0.16584605, 0.16801988, 0.27467, 0.16584605, 0.032019332, 0.032019332, 0.03266526, 0.032019332], 'C': [0.27394024, 0.614929, 0.49208787, 0.27383596, 0.9271388, 0.03653625, 0.037524186, 0.03653625], 'D': [0.63940144, 0.39265558, -0.13237914, -0.13237914, -0.19178307, -0.063766345, -0.19178307, -0.1998911], 'E': [1.6752424, 1.7658281, 0.87193537, 0.6394053, 0.73548913, 0.12432637, 0.12580438, 0.38651872], 'F': [-0.14359237, -0.14292242, -0.14361465, -0.14378239, 0.3467977, -0.40233213, -0.40208614, -0.4021718], 'G': [-0.2467654, 0.8191514, -0.2467654, -0.24764727, -0.34842074, -0.346546, -0.346546, 0.14414781], 'H': [-0.19641188, 0.3971681, 0.07294911, -0.36902112, -0.30496883, -0.30366325, 0.37729037, -0.3039636], 'I': [-0.36268777, 0.050759733, -0.3620067, -0.016449511, 0.07535602, -0.10393514, -0.10298226, 0.41284], 'J': [-0.046037797, -0.045171198, -0.04468745, -0.04630487, -

In [14]:
# Moirai multivariate
model = moirai_maker("moirai", gluon_multi)
predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data_multi.input)

input_it = iter(test_data_multi.input)
label_it = iter(test_data_multi.label)
forecast_it = iter(forecasts)

#inp = next(input_it)
#label = next(label_it)
#forecast = next(forecast_it)
results_dict = dict()
letter_sequence = "ABCDEFGHIJ"
for letter in letter_sequence:
    results_dict[letter] = []
    
for fore in forecast_it:
    #print(fore)
    for letter in letter_sequence:
        if fore.item_id == letter:
            #print(fore.samples)
            results_dict[letter].extend(np.median(fore.samples,axis=0))


print(results_dict)

{'A': [-0.012604862, -0.52203995, 0.9761853, 1.069562, -0.33709297, 0.30398068, 0.50754076, -0.5211502], 'B': [0.3004784, 0.75410986, -0.077502325, 0.36798602, 0.16729516, -0.9232799, 0.5867167, 0.16634227], 'C': [-0.067326754, 0.5246322, 0.90610874, -0.20859733, 0.46371543, 1.1043057, 1.3195828, -0.62426656], 'D': [-0.8738327, -0.13773657, -0.33924344, -1.0969563, 0.2909627, -1.0053365, -0.81486875, -0.26043648], 'E': [0.7745171, 0.18525466, 1.6390488, 0.98968494, -0.07147647, 0.11124915, -0.25061038, 0.00346639], 'F': [-0.30573457, -0.55762756, 1.283043, 0.20862092, 1.7236003, -0.55535084, -0.08466321, 1.0975668], 'G': [0.68304765, -0.54325235, 0.92590034, 1.4326, 0.45656186, -0.9413659, 0.25737196, -0.6638031], 'H': [-1.4825187, -0.3117496, -0.6818285, -0.5959078, -0.33135223, 2.5170393, 0.12852085, -0.014930159], 'I': [-0.21329978, -0.5170153, 0.407617, -0.9179652, -0.44414842, 0.23002039, 0.53676814, -0.4013853], 'J': [-0.2919862, 0.06928943, -0.39902785, 0.25311056, -0.13549854, 

In [15]:
# Moirai_MoE univariate
model = moirai_maker("moirai_moe", gluon_uni)
predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data_uni.input)

input_it = iter(test_data_uni.input)
label_it = iter(test_data_uni.label)
forecast_it = iter(forecasts)

#inp = next(input_it)
#label = next(label_it)
#forecast = next(forecast_it)
results_dict = dict()
letter_sequence = "A"
for letter in letter_sequence:
    results_dict[letter] = []
    
for fore in forecast_it:
    #print(fore)
    for letter in letter_sequence:
        if fore.item_id == letter:
            #print(fore.samples)
            results_dict[letter].extend(np.median(fore.samples,axis=0))


print(results_dict)

{'A': [0.45762113, 0.45759442, 0.45762113, 0.45762113, 0.29026833, 0.681697, 0.5874635, 0.812534]}


In [16]:
# Moirai univariate
model = moirai_maker("moirai", gluon_uni)
predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data_uni.input)

input_it = iter(test_data_uni.input)
label_it = iter(test_data_uni.label)
forecast_it = iter(forecasts)

#inp = next(input_it)
#label = next(label_it)
#forecast = next(forecast_it)
results_dict = dict()
letter_sequence = "A"
for letter in letter_sequence:
    results_dict[letter] = []
    
for fore in forecast_it:
    #print(fore)
    for letter in letter_sequence:
        if fore.item_id == letter:
            #print(fore.samples)
            results_dict[letter].extend(np.median(fore.samples,axis=0))


print(results_dict)

{'A': [0.29040825, 0.24619173, 0.6819386, 0.2566449, -0.6725775, -0.38204083, -0.5746957, -1.1273104]}
